In [1]:
# !pip install datasets
# !pip install transformers
# !pip install peft
# !pip install evaluate
# !pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
# !pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
# !pip install -U "transformers>=4.41" accelerate safetensors
# !pip install -U bitsandbytes
!pip install -U evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [2]:
!pip install -U fsspec aiohttp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.10.0
    Uninstalling fsspec-2025.10.0:
      Successfully uninstalled fsspec-2025.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.1 requires fsspec[http]<=2025.10.0,>=2023.1.0, but you have fsspec 2025.12.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
gradio 5.49.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.5 which is incompatible.
bigframes 2.26.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you 

In [3]:
!pip install --upgrade --no-cache-dir git+https://github.com/Shannu3766/bi_influence.git@version_1

  Cloning https://github.com/Shannu3766/bi_influence.git (to revision version_1) to /tmp/pip-req-build-pi9g0nhy
  Running command git clone --filter=blob:none --quiet https://github.com/Shannu3766/bi_influence.git /tmp/pip-req-build-pi9g0nhy
  Running command git checkout -b version_1 --track origin/version_1
  Switched to a new branch 'version_1'
  Branch 'version_1' set up to track remote branch 'version_1' from 'origin'.
  Resolved https://github.com/Shannu3766/bi_influence.git to commit 4dc6f1ccec6c2ed8210d2b8da51a42dd1172dce8
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 8.5 MB/s eta 0:00:00
  Created wheel for adaptive_lora: filename=adaptive_lora-2.1.0-py3-none-any.whl size=5843 sha256=38c71c9f8e21f01b169836d552d2d4f78114a63137665a2b41a304c17015ebaa
  Stored in directory: /tmp/pip-ephem-wheel-cache-mt_cay74/wheels/8d/29/cf/be29446b489124e88897058c28dfe358ba113f9b57f04c0ba4
Successfully built adaptive_lora
  Attempting uninsta

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:

import warnings
warnings.filterwarnings("ignore")
import random
import numpy as np
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model
import evaluate

2025-12-17 19:42:58.038285: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766000578.259263      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766000578.320003      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766000578.843402      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766000578.843449      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766000578.843452      24 computation_placer.cc:177] computation placer alr

In [6]:
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
OUTPUT_DIR = "./boolq_lora_tinyllama_adaptive"
BATCH_SIZE = 16         
EVAL_BATCH = 32
NUM_EPOCHS = 3
LR = 1e-4
SEED = 42
MAX_LENGTH = 256
LORA_R = 6
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
TARGET_MODULES =["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
ADAPTIVE_TAU = 0.9

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)

In [8]:
from datasets import load_dataset
import time

def load_boolq_with_retry(max_retries=None, wait=2):
    attempt = 0

    while True:
        try:
            ds = load_dataset("boolq")
            print(f"✓ Dataset 'boolq' loaded successfully after {attempt} attempts.")
            return ds

        except Exception as e:
            attempt += 1
            wait_time = wait * min(5, attempt)  # exponential backoff with cap

            print(f"[Attempt {attempt}] Failed to load 'boolq': {e}")
            print(f"Retrying in {wait_time} seconds...\n")

            # Stop if max retries exceeded
            if max_retries is not None and attempt >= max_retries:
                print("✗ Maximum retries reached. Raising error.")
                raise e

            time.sleep(wait_time)
dataset = load_boolq_with_retry(max_retries=10)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.69M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9427 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3270 [00:00<?, ? examples/s]

✓ Dataset 'boolq' loaded successfully after 0 attempts.


In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 9427
    })
    validation: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 3270
    })
})

In [10]:

train_ds = dataset["train"]
val_ds = dataset["validation"]
print("Train size:", len(train_ds))
print("Validation (test) size:", len(val_ds))

Train size: 9427
Validation (test) size: 3270


In [11]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [12]:
def clean_text(s):
    if s is None:
        return ""
    return " ".join(str(s).strip().split())


def preprocess(batch):
    questions = [clean_text(q) for q in batch["question"]]
    passages  = [clean_text(p) for p in batch["passage"]]

    reformatted_questions = [
        f"Question: {q}"
        for q in questions
    ]

    reformatted_passages = [
        f"Passage: {p}"
        for p in passages
    ]

    toks = tokenizer(
        reformatted_questions,
        reformatted_passages,
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length",  # you can change to False + DataCollator if desired
    )

    # BoolQ labels: answer=True → entailment (1), answer=False → non-entailment (0)
    toks["labels"] = [1 if ans else 0 for ans in batch["answer"]]

    return toks


print("Preprocessing datasets...")
train_ds = train_ds.map(preprocess, batched=True)
val_ds = val_ds.map(preprocess, batched=True)

Preprocessing datasets...


Map:   0%|          | 0/9427 [00:00<?, ? examples/s]

Map:   0%|          | 0/3270 [00:00<?, ? examples/s]

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

cols = ["input_ids", "attention_mask", "labels"]
train_ds.set_format(type="torch", columns=cols)
val_ds.set_format(type="torch", columns=cols)

train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=data_collator)
val_dataloader = DataLoader(val_ds, batch_size=EVAL_BATCH, shuffle=False, collate_fn=data_collator)

In [14]:
print("Model started Loading")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
    device_map="auto",
    torch_dtype=torch.bfloat16, 
    trust_remote_code=True
)

Model started Loading


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
print("Model Loaded")

Model Loaded


In [16]:
model.config.pad_token_id = tokenizer.pad_token_id

In [17]:
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="SEQ_CLS",
)
model = get_peft_model(model, lora_config)

In [18]:
try:
    model.print_trainable_parameters()
except Exception:
    trainable, total = 0, 0
    for _, p in model.named_parameters():
        total += p.numel()
        if p.requires_grad:
            trainable += p.numel()
    print(f"Trainable params: {trainable:,} / {total:,} ({100*trainable/total:.4f}%)")
    

# --- METRICS & CALLBACKS ---
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=preds, references=labels)
    return {"accuracy": acc["accuracy"]}


trainable params: 4,734,976 || all params: 1,039,251,456 || trainable%: 0.4556


In [19]:
from peft.tuners.lora import LoraLayer

def get_lora_module_names(peft_model):
    names = []
    for name, module in peft_model.named_modules():
        if isinstance(module, LoraLayer):
            names.append(name)
    return names

lora_names = get_lora_module_names(model)   # `model` is your PEFT-wrapped model
print(f"🔢 Number of LoRA modules: {len(lora_names)}\n")

# for n in lora_names:
    # print(" ", n)
num_lora_modules = len(lora_names)
TOTAL_RANK_BUDGET = LORA_R * num_lora_modules

avg_rank_per_module = TOTAL_RANK_BUDGET / num_lora_modules
print(
    f"💡 With TOTAL_RANK_BUDGET={TOTAL_RANK_BUDGET} over "
    f"{num_lora_modules} modules, avg rank ≈ {avg_rank_per_module:.2f}"
)

🔢 Number of LoRA modules: 154

💡 With TOTAL_RANK_BUDGET=924 over 154 modules, avg rank ≈ 6.00


In [20]:
from adaptive_lora.callbacks import AdaptiveLoRACallback
callback_val_loader = DataLoader(
    val_ds, 
    batch_size=16, 
    shuffle=False, 
    collate_fn=data_collator
)
adaptive_callback = AdaptiveLoRACallback(
    total_rank=TOTAL_RANK_BUDGET, 
    tau=ADAPTIVE_TAU,
    val_dataloader=callback_val_loader,
    verbose=True,
    validate_batch_size=8,
)

In [21]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=200,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    seed=SEED,                      
    gradient_accumulation_steps=2,      
    fp16=False,                             
    bf16=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[adaptive_callback],
)

print("\nStarting training on BoolQ...")
trainer.train()


Starting training on BoolQ...

--- AdaptiveLoRA: Preparing ranks for Epoch 1 ---
Computing BI importance scores (pre-training)...


Computing BI scores:   0%|          | 0/205 [00:00<?, ?it/s]

Allocating new ranks based on BI scores...
Applying new ranks to LoRA modules for this epoch...
  - base_model.model.model.layers.0.self_attn.q_proj: r=6 (Unchanged, Score: 0.9908)
  - base_model.model.model.layers.0.self_attn.k_proj: r=6 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.0.self_attn.v_proj: r=6 (Unchanged, Score: 0.9963)
  - base_model.model.model.layers.0.self_attn.o_proj: r=6 (Unchanged, Score: 0.9855)
  - base_model.model.model.layers.0.mlp.gate_proj: r=6 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.0.mlp.up_proj: r=6 (Unchanged, Score: 0.9976)
  - base_model.model.model.layers.0.mlp.down_proj: r=6 (Unchanged, Score: 0.9933)
  - base_model.model.model.layers.1.self_attn.q_proj: r=6 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.1.self_attn.k_proj: r=6 → 5 (Score: 0.9741)
  - base_model.model.model.layers.1.self_attn.v_proj: r=6 (Unchanged, Score: 0.9995)
  - base_model.model.model.layers.1.self_attn.o_proj: r=6 (Unchanged, Sc

Epoch,Training Loss,Validation Loss,Accuracy
1,1.492200,0.641555,0.645872
2,1.177500,0.554443,0.728746
3,0.725900,0.558313,0.740061


📄 Epoch 1: Rank allocations logged to ./logs/adaptive_lora_epoch_logs.csv


--- AdaptiveLoRA: Preparing ranks for Epoch 2 ---
Computing BI importance scores (pre-training)...


Computing BI scores:   0%|          | 0/205 [00:00<?, ?it/s]

Allocating new ranks based on BI scores...
Applying new ranks to LoRA modules for this epoch...
  - base_model.model.model.layers.0.self_attn.q_proj: r=6 (Unchanged, Score: 0.9908)
  - base_model.model.model.layers.0.self_attn.k_proj: r=6 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.0.self_attn.v_proj: r=6 (Unchanged, Score: 0.9946)
  - base_model.model.model.layers.0.self_attn.o_proj: r=6 (Unchanged, Score: 0.9862)
  - base_model.model.model.layers.0.mlp.gate_proj: r=6 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.0.mlp.up_proj: r=6 (Unchanged, Score: 0.9967)
  - base_model.model.model.layers.0.mlp.down_proj: r=6 (Unchanged, Score: 0.9933)
  - base_model.model.model.layers.1.self_attn.q_proj: r=6 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.1.self_attn.k_proj: r=5 (Unchanged, Score: 0.9738)
  - base_model.model.model.layers.1.self_attn.v_proj: r=6 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.1.self_attn.o_proj: r=6 (Unchan

Computing BI scores:   0%|          | 0/205 [00:00<?, ?it/s]

Allocating new ranks based on BI scores...
Applying new ranks to LoRA modules for this epoch...
  - base_model.model.model.layers.0.self_attn.q_proj: r=6 (Unchanged, Score: 0.9908)
  - base_model.model.model.layers.0.self_attn.k_proj: r=6 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.0.self_attn.v_proj: r=6 (Unchanged, Score: 0.9944)
  - base_model.model.model.layers.0.self_attn.o_proj: r=6 (Unchanged, Score: 0.9853)
  - base_model.model.model.layers.0.mlp.gate_proj: r=6 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.0.mlp.up_proj: r=6 (Unchanged, Score: 0.9970)
  - base_model.model.model.layers.0.mlp.down_proj: r=6 (Unchanged, Score: 0.9933)
  - base_model.model.model.layers.1.self_attn.q_proj: r=6 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.1.self_attn.k_proj: r=5 (Unchanged, Score: 0.9734)
  - base_model.model.model.layers.1.self_attn.v_proj: r=6 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.1.self_attn.o_proj: r=6 (Unchan

TrainOutput(global_step=885, training_loss=1.054646684894454, metrics={'train_runtime': 21708.2011, 'train_samples_per_second': 1.303, 'train_steps_per_second': 0.041, 'total_flos': 4.230512799095194e+16, 'train_loss': 1.054646684894454, 'epoch': 3.0})

In [22]:
print("\nEvaluating final model...")
eval_result = trainer.evaluate(eval_dataset=val_ds)
test_accuracy = eval_result.get("eval_accuracy") or eval_result.get("accuracy")

print("\n=== Final testing (validation) results ===")
for k, v in eval_result.items():
    print(f"{k}: {v}")

if test_accuracy is not None:
    print(f"\nTest (validation) Accuracy: {test_accuracy:.4f}")
else:
    print("\nAccuracy not found in eval results. Full eval_result shown above.")

model.save_pretrained(OUTPUT_DIR)
print(f"\nLoRA adapter + model saved to {OUTPUT_DIR}")


Evaluating final model...



=== Final testing (validation) results ===
eval_loss: 0.5583133697509766
eval_accuracy: 0.7400611620795107
eval_runtime: 839.4203
eval_samples_per_second: 3.896
eval_steps_per_second: 0.123
epoch: 3.0

Test (validation) Accuracy: 0.7401

LoRA adapter + model saved to ./boolq_lora_tinyllama_adaptive
